# **Total Market Capitalization to GDP (Buffett Indicator)**
by Phuoc Le<br>
Last update: September 16, 2020

## 1. Set up

Importing all the necessary libraries for the notebook.

In [1]:
import pandas_datareader as pdr
import pandas as pd
import datetime
import plotly.express as px

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# 2. Pull data from FRED and calculate the ratio.
The indicator will be calculated as follows:

*   Pull quarterly GDP data from FRED
*   Pull the Wilshire 5000 Full Cap Price Index from FRED
*   Convert daily series to quarterly series
*   Calculate the index as [Quarterly Market Cap] / [GDP]

In [2]:
%%capture
# Start and End in yyyy, mm, dd format
start = datetime.datetime (1971, 1, 1)
end = datetime.datetime (2020, 12, 31)

### GET MARKET CAP DATA ###
data_series = "WILL5000PRFC"
market_cap = pdr.DataReader(data_series, 'fred', start, end)

# Drop NaN values from market_cap dataframe
market_cap = market_cap.dropna()

# Conver dataframe index to datetime object
market_cap.index = pd.to_datetime(market_cap.index)

### GET GDP DATA ###
data_series = "GDP"
gdp = pdr.DataReader(data_series, 'fred', start, end)
# Drop NaN values from gdp dataframe
gdp = gdp.dropna()

# Conver dataframe index to datetime object
gdp.index = pd.to_datetime(gdp.index)

# Create quarterly market_cap
market_cap_quarterly = pd.DataFrame()

market_cap_quarterly = market_cap
market_cap_quarterly.resample("3M").last()

# Merge the two dataframes using pandas' merge() on their indices
market_cap_to_gdp = pd.merge(market_cap_quarterly, gdp, left_index=True, right_index=True)

# Create Date column so we can use it as the lable in chart
market_cap_to_gdp["Date"] = market_cap_to_gdp.index

# Multiply the Wilshire 5000 Full Cap index by $1.19 billion to set it to 1980's USD per Wilshire's notes.
market_cap_to_gdp["WILL5000PRFC"] = market_cap_to_gdp["WILL5000PRFC"] * 1190000000

# Calculate the indicator as market cap / GDP
market_cap_to_gdp["Buffett Indicator"] = market_cap_to_gdp["WILL5000PRFC"] / market_cap_to_gdp["GDP"]

# 3. Plot the indicator using Plotly
Plot it using Plotly Express and style it to make it look pretty.

In [3]:
indicator_chart = px.area(market_cap_to_gdp, x='Date', y='Buffett Indicator',
    title='Stock Market Capitalization to GDP aka "The Buffett Indicator"',
    color_discrete_sequence=["#4499dd"],
    labels="",
    width=1250,
    height=600)

indicator_chart.update_layout(margin=dict(l=100,r=40,t=80, b=40),
    plot_bgcolor = "#eeeeee",
    paper_bgcolor="#f8f8f8") 

indicator_chart.show()